# Do injections and PE
... in parallel. We have two options for parallelizing:
1. Use SLURM (to submit jobs to a cluster), or
1. Use `multiprocessing` (to use the cores of the machine this notebook is running on).

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

import sys
sys.path.append('../..')

import multiprocessing
import pandas as pd
import itertools
import numpy as np

from cogwheel.validation import inference
from cogwheel.validation import config
from cogwheel.validation import generate_injections

from cogwheel import gw_plotting
from cogwheel import gw_prior
from cogwheel import sampling

In [ ]:
injection_ids = list(
    itertools.product(range(len(config.MCHIRP_RANGES)),
                      range(config.N_INJECTIONS_ABOVE_THRESHOLD)))

rundirs = [inference._get_rundir(i_inj_set, i_sample)
           for i_inj_set, i_sample in injection_ids]

## Option 0: run a single one
(Probably worth doing this first, to check for bugs)

In [ ]:
inference.main(rundirs[0])

## Option 1: SLURM

In [ ]:
for i_inj_set, i_sample in injection_ids[1:]:
    inference.submit_slurm(i_inj_set, i_sample)

In [ ]:
!squeue --me

## Option 2: `multiprocessing`

In [ ]:
with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
    pool.map(inference.main, rundirs)